# 1.复习上课内容

# 2. 回答一下理论题目

1. What is independent assumption in Naive bayes ?

样本中各维度特征 $$ x_1, x_2, x_3, x_4, ... ,x_n $$相互独立，即: $$ P(x_1, x_2, x_3, x_4, ... ,x_n|y_k) = \prod_{i=1}^n P(x_i|y_k) $$
  
2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

根据贝叶斯公式，$$ P(class|x) = P(x|class)*P(class) $$
MAP: 对于x属于哪一类，即: 求$$argmax(P(class|x)) = argmax(P(x|class)*P(class))$$
ML: 假设先验概率P(class)为均匀分布，那么$$argmax(P(class|x)) = argmax(P(x|class) $$
对于x属于哪一类预估的标准是：该类使得出现x的概率最大。


3. What is support vector in SVM?

最靠近最优分类超平面两侧的样本点（最优超平面对应的两侧分类面穿过的点），为支持向量。


4. What is the intuition behind SVM ?

在所有样本点中找到一个分类超平面，使所有样本点（包括正反分类）距离超平面尽可能远。


5. Shortly describ what 'random' means in random forest ?

随机取得样本、随机选择某些特征。


6. What cariterion does XGBoost use to find the best split point in a tree ?

使得增益Gain函数的值最大。


# 3. Practial part

In [1]:
###对新闻语料进行分词处理，生成训练和测试数据集；新华社新闻Label为“Y”，非新华社新闻为“N”
import jieba
#import re
import pandas as pd
from sklearn.model_selection import train_test_split


def dataset(file_path):
    x_trainings = []
    y_labels = []
    csv = pd.read_csv(file_path, encoding='gb18030')
    for line in csv.values:
        x_trainings.append(line[3])
        if line[2] == '新华社':
            y_labels.append('Y')
        else:
            y_labels.append('N')
    return x_trainings, y_labels

file_path = r'C:\Users\dell\Desktop\others\NLP-lecture10\sqlResult_1558435.csv'
x_trainings, y_labels = dataset(file_path)[0], dataset(file_path)[1]

stoplist = {}.fromkeys([line.strip() for line in open('stopwords.txt', encoding='utf-8')])

def fenci(trainning_data):
    #trainning_data = re.findall(r'[\u4e00-\u9fa5]',trainning_data)
    seg = [word for word in jieba.cut(str(trainning_data)) if word not in stoplist]
    text_string = ' '.join(seg)
    return text_string

def x_trainning_fenci(x_trainning):
    X_trainning = []
    for text in x_trainning:
        X_trainning.append(fenci(str(text)))
    return X_trainning

X_trainning = x_trainning_fenci(x_trainings)

X_train, X_test, y_train, y_test = train_test_split(X_trainning, y_labels, random_state=1)

#print(X_train[:10])
#print(y_train[:10])
    

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Temp\jieba.cache
Loading model cost 1.010 seconds.
Prefix dict has been built succesfully.


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

DTC_classifier = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=10)
DTC_classifier.fit(TV.transform(X_train), y_train)
#print(DTC_classifier.score(TV.transform(X_test), y_test))
y_predict = DTC_classifier.predict(TV.transform(X_test))
results = classification_report(y_test, y_predict, digits=4)
#dict_1 = results.to_list(orient='records')
print(results)


NameError: name 'TV' is not defined

In [3]:
###使用sklearn-TfidfVectorizer进行text向量转换，并建立classifier模型
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import datetime

stoplist = {}.fromkeys([line.strip() for line in open('stopwords.txt', encoding='utf-8')])
TV = TfidfVectorizer(stop_words=stoplist, max_features=10000, smooth_idf=True, lowercase = False)
TV.fit(X_trainning)
X_train_vector = TV.transform(X_train)#.toarray()
X_test_vector = TV.transform(X_test)#.toarray()

def bi_classification(x_trainning, y_trainning, x_test, y_test):
    models=[]
    models_params = {}
    models.append(('MultinomialNB', MultinomialNB()))
    #models.append(('GaussianNB', GaussianNB()))
    models.append(('BernoulliNB', BernoulliNB()))
    models.append(('KNN', KNeighborsClassifier(n_neighbors=6)))
    models.append(('LogisticRegre', LogisticRegression(penalty='l1', tol=0.01)))
    models.append(('SVM', svm.SVC(kernel='linear', probability=True, C=1000)))
    models.append(('Decision_Tree', DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=100)))
    models.append(('RandomForest', RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=100)))
    models.append(('xgbc', XGBClassifier()))
    
    for classifier_method, classifier in models: 
        model_params = []
        start_time = datetime.datetime.now()
        #if classifier_method == 'GaussianNB':
            #classifier.fit(x_trainning.toarray(), y_trainning)
        #else:
        classifier.fit(x_trainning, y_trainning)
        end_time = datetime.datetime.now()
        run_time = (end_time - start_time).microseconds
        y_predict = classifier.predict(x_test)
        Accuracy = accuracy_score(y_test, y_predict)
        precision = precision_score(y_test, y_predict, pos_label='Y')
        recall = recall_score(y_test, y_predict, pos_label='Y')
        F1_score = f1_score(y_test, y_predict, pos_label='Y')
        model_params.extend([Accuracy, precision, recall, F1_score, run_time])
        models_params[classifier_method] = model_params
        
    return models_params

models_results = bi_classification(X_train_vector, y_train, X_test_vector, y_test)
#print(bi_classification(X_train_vector, y_train, X_test_vector, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [4]:
###对比各classifier主要评价参数和运行时间
pd.DataFrame.from_dict(models_results, orient='index', columns=['Accuracy', 'Y_Precision', 'Recall', 'F1_score', 'Run_time'])

,Accuracy,Y_Precision,Recall,F1_score,Run_time
MultinomialNB,0.911574,0.975876,0.922214,0.948286,75004
BernoulliNB,0.810472,0.940418,0.837471,0.885964,125007
KNN,0.919297,0.937271,0.973343,0.954967,42002
LogisticRegre,0.988662,0.989673,0.997512,0.993577,456312
SVM,0.974691,0.983079,0.988220,0.985643,477593
Decision_Tree,0.984734,0.988096,0.994618,0.991346,151610
RandomForest,0.971298,0.969307,0.998985,0.983922,394484
xgbc,0.982636,0.990398,0.989845,0.990122,725018
